In [1]:
import regex as re
import pandas as pd
import numpy as np

#### convertirlo en una linea

### Convertirlos desde la secuencia
<img src="canonical.png">

In [2]:
r = pd.read_csv('../miR_Family_Info_targetScan_2016.txt', sep='\t', index_col=3)
miRseqFC = pd.DataFrame(r)



def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'U':'A'}
    return ''.join([complement[base] for base in dna[::-1]])


In [3]:
lista_miRs = list(miRseqFC[(miRseqFC['Species ID'] == 9606) & (miRseqFC['Family Conservation?'] == 2)].index)

#Oncomirs_todos
#lista_miRs = ['hsa-miR-187-3p', 'hsa-miR-205-5p', 'hsa-miR-221-3p', 'hsa-miR-143-3p', 'hsa-miR-184', 'hsa-miR-133b', 'hsa-miR-490-3p', 'hsa-miR-136-3p', 'hsa-miR-23c','hsa-miR-183-5p.1', 'hsa-miR-126-3p.1','hsa-miR-17-5p', 'hsa-miR-17-3p', 'hsa-miR-19a-3p', 'hsa-miR-19b-3p', 'hsa-miR-20a-5p', 'hsa-miR-21-5p', 'hsa-miR-25-3p', 'hsa-miR-92a-3p', 'hsa-miR-93-5p', 'hsa-miR-96-5p', 'hsa-miR-103a-3p', 'hsa-miR-106a-5p', 'hsa-miR-148a-3p', 'hsa-miR-182-5p','hsa-miR-199b-5p', 'hsa-miR-210-3p', 'hsa-miR-30b-5p', 'hsa-miR-141-3p', 'hsa-miR-153-3p', 'hsa-miR-191-5p', 'hsa-miR-9-5p', 'hsa-miR-200c-3p', 'hsa-miR-106b-5p', 'hsa-miR-362-5p', 'hsa-miR-375', 'hsa-miR-342-3p', 'hsa-miR-20b-5p', 'hsa-miR-146b-5p', 'hsa-miR-500a-3p', 'hsa-miR-660-5p', 'hsa-miR-425-5p', 'hsa-miR-20a-3p', 'hsa-miR-93-3p', 'hsa-miR-7-1-3p', 'hsa-miR-141-5p', 'hsa-miR-200c-5p', 'hsa-miR-423-5p', 'hsa-miR-146b-3p', 'hsa-miR-501-3p', 'hsa-miR-629-5p', 'hsa-miR-708-5p', 'hsa-miR-708-3p', 'hsa-miR-1307-3p', 'hsa-miR-3065-3p', 'hsa-miR-3607-3p']
#supresores los primeros hasta hsa-miR-23c

#'hsa-miR-183-5p.1','hsa-miR-126-3p.1' #problemas con el .1 ya lo solucioné

<img src="ejemplolet7.PNG">

In [4]:
#ahora es la secuencia de hsa-let-7a-5p
mir122 = 'UGAGGUAGUAGGUUGUAUAGUU'
recmir122 = reverse_complement(mir122)
seed6122 = mir122[1:7]
seed6compl122 = reverse_complement(seed6122)

extra13_16122 = recmir122[-16:-12]
extra15_19122 = recmir122[-19:-14]

print(seed6122)
print(seed6compl122)
print(extra13_16122 )
print(extra15_19122)

GAGGUA
TACCTC
ACAA
TATAC


In [5]:
mir_seq = miRseqFC.loc[lista_miRs,:]

#### Vamos a hacer tal cual Targetscan

In [6]:
cont = 0
dict_regexp = {}


for index,row in mir_seq.iterrows():
    cont += 1
    rev = reverse_complement(row['Mature sequence'])
    #seed
    seed6 = row['Mature sequence'][1:7]
    seed7 = row['Mature sequence'][1:8]
    #seed complement
    seed6compl = reverse_complement(seed6)
    seed7compl = reverse_complement(seed7)
    #next nucleotide
    no7 = reverse_complement(row['Mature sequence'][7])
    no8 = reverse_complement(row['Mature sequence'][8])
    
    #print(row['Mature sequence'][8])
    
    extra4_13_16 = rev[-16:-12]
    
    extra4_15_19 = rev[-19:-14]
    
    #print(index, row['Mature sequence'])
    #print(seed6)
    #print(seed6compl)
    #print(extra4_13_16)
    #print(extra4_15_19)
    
    #print(extra4_13_16)
    
    site_6mer = '[^'+ str(no7) + ']' + seed6compl + '[^A]'
    site_7merA1 = '[^'+ str(no7) + ']' + seed6compl + 'A'
    site_7merm8 = '[^'+ str(no8) + ']' + seed7compl + '[^A]'
    site_8mer = '[^'+ str(no8) + ']' + seed7compl + 'A'
    
    
    site_6mer_suppl = '(' + str(extra4_13_16) +')' +'((?:(?!'+ str(extra4_13_16) + ').){0,14}?)' +'([^'+ str(no7) + ']' + seed6compl + '[^A]' + ')'
    site_7merA1_suppl = '(' + str(extra4_13_16) +')' +'((?:(?!'+ str(extra4_13_16) + ').){0,14}?)' +'([^'+ str(no7) + ']' + seed6compl + 'A' + ')'
    site_7merm8_suppl = '(' + str(extra4_13_16) +')' +'((?:(?!'+ str(extra4_13_16) + ').){0,14}?)' +'([^'+ str(no8) + ']' + seed7compl + '[^A]' + ')'
    site_8mer_suppl = '(' + str(extra4_13_16) +')' +'((?:(?!'+ str(extra4_13_16) + ').){0,14}?)' +'([^'+ str(no8) + ']' + seed7compl + 'A' + ')'
    
    dict_regexp[index] = [site_6mer, site_7merA1, site_7merm8, site_8mer, site_6mer_suppl, site_7merA1_suppl, site_7merm8_suppl, site_8mer_suppl]

In [7]:

f = open('diccionario_REGEXP.txt', 'w')
f.write(str(dict_regexp))
f.close()

In [8]:
diccionario_para_Guille = {}
for i in dict_regexp:
    diccionario_para_Guille[i] = dict_regexp[i][0]

f = open('diccionario_para_guille.txt', 'w')
f.write(str(diccionario_para_Guille))
f.close()

In [9]:
def grabarfile(linea1, linea2, miR, motivo, NumMotivo , seqmedia, seqmediaLargo, arch):
                    
    linea1 = line1.strip()
    linea2 = line2.strip()
    miR = miR.strip()
    motivo = motivo.strip()
    
    arch.write(linea1)
    arch.write('\t')
    arch.write(linea2)
    arch.write('\t')
    arch.write(miR)
    arch.write('\t')
    arch.write(motivo)
    arch.write('\t')
    arch.write(str(NumMotivo))
    arch.write('\t')
    arch.write(seqmedia)
    arch.write('\t')
    arch.write(str(seqmediaLargo))
    arch.write('\n')   


In [10]:

archivoTODAS = open('ExpresionesRegulares_TargetScanCANONICAL_3UTR.csv', 'w')


for mir in dict_regexp:
    
    prog6mer = re.compile(dict_regexp[mir][0])
    prog7merA1 = re.compile(dict_regexp[mir][1])
    prog7merm8 = re.compile(dict_regexp[mir][2])
    prog8mer = re.compile(dict_regexp[mir][3])
    
    prog6mer_SUPPL = re.compile(dict_regexp[mir][4])
    prog7merA1_SUPPL = re.compile(dict_regexp[mir][5])
    prog7merm8_SUPPL = re.compile(dict_regexp[mir][6])
    prog8mer_SUPPL = re.compile(dict_regexp[mir][7])
    
    ciento = open('../martquery_0427225317_527_unaLinea.txt', 'r') #one-line fasta file of sequence to search
    
    line1 = ciento.readline()
    line2 = ciento.readline()
    
    while line1:
        
        if line1.startswith('>protein_coding'): #== True:
        

            resultado6mer = re.findall(prog6mer, line2)
            resultado7merA1 = re.findall(prog7merA1, line2)
            resultado7merm8 = re.findall(prog7merm8, line2)
            resultado8mer = re.findall(prog8mer, line2)
            
            resultado6mer_SUPPL = re.findall(prog6mer_SUPPL, line2)
            resultado7merA1_SUPPL = re.findall(prog7merA1_SUPPL, line2)
            resultado7merm8_SUPP = re.findall(prog7merm8_SUPPL, line2)
            resultado8mer_SUPPL = re.findall(prog8mer_SUPPL, line2)

            
            #-------------------------
            #Primero las seeds solas
            #-------------------------
            if len(resultado6mer) != 0:
                
                for regExp in resultado6mer:                    

                    grabarfile(line1, line2, mir, '6mer_site', len(resultado6mer), '6mer_site' , 0 ,archivoTODAS)
                
            if len(resultado7merA1) != 0:
                
                for regExp in resultado7merA1:
                    
                    grabarfile(line1, line2, mir, '7merA1_site', len(resultado7merA1), '7merA1_site' , 0 ,archivoTODAS)
                
            if len(resultado7merm8) != 0:
                
                for regExp in resultado7merm8:
                    
                    grabarfile(line1, line2, mir, '7merm8_site', len(resultado7merm8), '7merm8_site', 0 , archivoTODAS)
                    
            if len(resultado8mer) != 0:
                
                for regExp in resultado8mer:
                    
                    grabarfile(line1, line2, mir, '8mer_site', len(resultado8mer), '8mer_site', 0 , archivoTODAS)
                    
                    
            #------------------------
            #Ahora las suplementarias
            #------------------------
            if len(resultado6mer_SUPPL) != 0:
                
                for regExp in resultado6mer_SUPPL:                    

                    grabarfile(line1, line2, mir, '6mer_site_SUPPL', len(resultado6mer_SUPPL), resultado6mer_SUPPL[0][1], len(resultado6mer_SUPPL[0][1]) ,archivoTODAS)
                
            if len(resultado7merA1_SUPPL) != 0:
                
                for regExp in resultado7merA1_SUPPL:
                    
                    grabarfile(line1, line2, mir, '7merA1_site_SUPPL', len(resultado7merA1_SUPPL), resultado7merA1_SUPPL[0][1], len(resultado7merA1_SUPPL[0][1]) ,archivoTODAS)
                
            if len(resultado7merm8_SUPP) != 0:
                
                for regExp in resultado7merm8_SUPP:
                    
                    grabarfile(line1, line2, mir, '7merm8_site_SUPPL', len(resultado7merm8_SUPP), resultado7merm8_SUPP[0][1], len(resultado7merm8_SUPP[0][1]) , archivoTODAS)
                    
            if len(resultado8mer_SUPPL) != 0:
                
                for regExp in resultado8mer_SUPPL:
                    
                    grabarfile(line1, line2, mir, '8mer_site_SUPPL', len(resultado8mer_SUPPL), resultado8mer_SUPPL[0][1], len(resultado8mer_SUPPL[0][1]) , archivoTODAS)

                
        line1 = ciento.readline()
        line2 = ciento.readline()
        
        
        

archivoTODAS.close()
   
   



In [11]:
df_3UTR = pd.read_csv('ExpresionesRegulares_TargetScanCANONICAL_3UTR.csv', sep='\t',\
                      names=['gene_name', 'seq','microRNA','site', 'Number_sites','bridge','bridge_length'])

#df_3UTR.set_index('gene_name') 
#df_3UTR.drop('seq', axis=1, inplace=True)

#df_3UTR.to_csv('ExpresionesRegulares_TargetScanCANONICAL_3UTR_sinSEQ.csv', sep='\t')

In [12]:
print('final')

final
